In [1]:
import sys
sys.path.append("..")

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

import numpy as np
import pandas as pd
from tabulate import tabulate

# Trainer
from src.trainer.trainer import Trainer

# Models
from src.models.mlp import MLP

## CIFAR10 dataset

In [2]:
root_path = "../data"
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor()
])
transform_test = transforms.ToTensor()

full_train = datasets.CIFAR10(root=root_path, train=True, download=True, transform=transform_train)

train_size = int(0.9 * len(full_train))
val_size = len(full_train) - train_size
train_set, val_set = random_split(full_train, [train_size, val_size])
test_set = datasets.CIFAR10(root=root_path, train=False, download=True, transform=transform_test)


batch_size = 128
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

In [3]:
img, label = full_train[0]
print(img.shape)

torch.Size([3, 32, 32])


## Table generation

In [7]:
# Init training variables
num_epochs = 10
num_iterations = 3
log_dir = "../logs"
accuracies = {
    "mlp": []
}

# ----- Init model paramerters -----

## MLP parameters:
input_size = 3 * 32 * 32
output_size = 10
hidden_sizes = [128, 64]

## ---------------------------------

for it in range(num_iterations):
    
    # ----- Models initialization -----
    mlp_model = MLP(
        input_size = input_size,
        hidden_sizes = hidden_sizes,
        output_size = output_size
    )

    # ---------------------------------

    # ----- Other parameter initialization -----
    mlp_criterion = nn.CrossEntropyLoss()
    mlp_optimizer = optim.SGD(mlp_model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
    mlp_scheduler = optim.lr_scheduler.MultiStepLR(mlp_optimizer, milestones=[200, 250, 300], gamma=0.1)

    # ------------------------------------------

    # Generate lists
    models = {
        "mlp": mlp_model
    }
    criterions = [mlp_criterion]
    optimizers = [mlp_optimizer]
    schedulers = [mlp_scheduler]

    # -------------------------------------------
    
    trainer = Trainer(
        models = models,
        optimizers = optimizers,
        criterions = criterions,
        schedulers = schedulers,
        log_dir = log_dir
    )

    # Train models on dataset
    print("Training models...")
    trainer.train(
        num_epochs = num_epochs,
        train_loader = train_loader,
        val_loader = val_loader,
    )

    # Evaluate models on dataset
    print("Testing models...")
    test_accuracies = trainer.evaluate(test_loader = test_loader)

    for model_name, acc in test_accuracies.items():
        accuracies[model_name].append(acc)

# get final statistics
final_stats = {
    model_name: {
        "% Test error" : (1 - float(np.mean(values))) * 100,
        "% std" : float(np.std(values)) * 100,
        "Num Parameters" : sum(p.numel() for p in models[model_name].parameters()),
    }
    for model_name, values in accuracies.items()
}

Using device: cuda
Current GPU: NVIDIA GeForce RTX 4070 Laptop GPU
Training models...
Training model : mlp


Training Epochs: 100%|██████████████████████████████████████████████████████████████████| 10/10 [00:24<00:00,  2.47s/it, Train Acc=0.2336, Val Acc=0.2454, Train Loss=2.0266, Val Loss=2.0364]

Testing models...


Using device: cuda
Current GPU: NVIDIA GeForce RTX 4070 Laptop GPU
Training models...
Training model : mlp


Training Epochs: 100%|██████████████████████████████████████████████████████████████████| 10/10 [00:24<00:00,  2.46s/it, Train Acc=0.2521, Val Acc=0.2730, Train Loss=1.9718, Val Loss=1.9753]

Testing models...


Using device: cuda
Current GPU: NVIDIA GeForce RTX 4070 Laptop GPU
Training models...
Training model : mlp


Training Epochs: 100%|██████████████████████████████████████████████████████████████████| 10/10 [00:24<00:00,  2.46s/it, Train Acc=0.2191, Val Acc=0.2112, Train Loss=2.0283, Val Loss=2.0244]

Testing models...


## Table

In [8]:
df = pd.DataFrame.from_dict(final_stats, orient='index')
df = df.round(2)
print("📊 Model Accuracy Summary in CIFAR10\n")
print(tabulate(df, headers="keys", tablefmt="fancy_grid"))

📊 Model Accuracy Summary in CIFAR10

╒═════╤════════════════╤═════════╤══════════════════╕
│     │   % Test error │   % std │   Num Parameters │
╞═════╪════════════════╪═════════╪══════════════════╡
│ mlp │          74.52 │    2.38 │           402250 │
╘═════╧════════════════╧═════════╧══════════════════╛
